# SAR Image Colorization - Experiment Tracking & Comparison

## Overview
This notebook provides comprehensive experiment tracking and comparison for SAR image colorization models. It includes:

- **Experiment Logging**: Loading and analyzing logged results from TensorBoard, CSV, JSON
- **Model Comparison**: Side-by-side comparison of different model architectures
- **Training Curves**: Visualization of training progress and convergence
- **Hyperparameter Analysis**: Analysis of hyperparameter impact on performance
- **Best Model Selection**: Automated selection of best performing models
- **Reproducibility**: Ensuring experiment reproducibility and documentation

## Key Features:
1. **Experiment Management**: Comprehensive experiment tracking and organization
2. **Visualization Suite**: Advanced visualization of training progress and results
3. **Statistical Analysis**: Statistical analysis of experiment results
4. **Model Selection**: Automated model selection based on multiple criteria
5. **Reproducibility**: Ensuring experiment reproducibility and documentation

## Usage
1. Load experiment logs and results
2. Analyze training progress and convergence
3. Compare different model architectures
4. Select best models for deployment

## Dependencies
- `tensorboard` - TensorBoard log analysis
- `pandas` - Data analysis and manipulation
- `matplotlib` - Visualization
- `seaborn` - Statistical visualization
- `scikit-learn` - Statistical analysis


In [ ]:
# Import required libraries
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import pandas as pd
import json
import glob
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# TensorBoard imports
try:
    from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
    print("✅ TensorBoard support available")
except ImportError:
    print("⚠️ TensorBoard not available - some features will be limited")

# Add src to path for imports
sys.path.append('../src')

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("✅ Libraries imported successfully!")
print(f"📁 Current working directory: {os.getcwd()}")
print(f"🐍 Python version: {sys.version}")
print(f"📊 Pandas version: {pd.__version__}")
print(f"📈 Matplotlib version: {plt.matplotlib.__version__}")
print(f"🎨 Seaborn version: {sns.__version__}")


In [ ]:
# Configuration for experiment tracking
CONFIG = {
    'experiments_root': '../experiments',
    'logs_path': '../experiments/logs',
    'checkpoints_path': '../experiments/checkpoints',
    'results_path': '../experiments/results',
    'experiment_types': ['supervised', 'gan', 'adversarial'],
    'metrics_to_track': [
        'train_loss', 'val_loss', 'train_psnr', 'val_psnr',
        'train_ssim', 'val_ssim', 'learning_rate', 'epoch_time'
    ],
    'visualization_config': {
        'figure_size': (15, 10),
        'dpi': 100,
        'style': 'seaborn-v0_8',
        'color_palette': 'husl'
    }
}

print("🔧 Experiment Tracking Configuration:")
print(f"   Experiments root: {CONFIG['experiments_root']}")
print(f"   Logs path: {CONFIG['logs_path']}")
print(f"   Checkpoints path: {CONFIG['checkpoints_path']}")
print(f"   Results path: {CONFIG['results_path']}")
print(f"   Experiment types: {CONFIG['experiment_types']}")
print(f"   Metrics to track: {CONFIG['metrics_to_track']}")

# Verify experiment directories
print("\n🔍 Verifying experiment directories...")
for key, path in CONFIG.items():
    if 'path' in key and os.path.exists(path):
        if key == 'logs_path':
            # Check for TensorBoard logs
            tb_logs = glob.glob(os.path.join(path, '**/tensorboard/*'), recursive=True)
            print(f"✅ {key}: {path} ({len(tb_logs)} TensorBoard logs)")
        else:
            print(f"✅ {key}: {path}")
    elif 'path' in key:
        print(f"❌ {key}: {path} (not found)")

# Create results directory if it doesn't exist
os.makedirs(CONFIG['results_path'], exist_ok=True)
print(f"✅ Results directory: {CONFIG['results_path']}")


In [ ]:
# Experiment Data Loading
def load_experiment_data():
    """Load experiment data from various sources"""
    
    experiment_data = {}
    
    # Load TensorBoard logs if available
    if 'EventAccumulator' in globals():
        print("📊 Loading TensorBoard logs...")
        tb_logs = glob.glob(os.path.join(CONFIG['logs_path'], '**/tensorboard/*'), recursive=True)
        
        for log_path in tb_logs:
            try:
                # Extract experiment name from path
                exp_name = os.path.basename(os.path.dirname(log_path))
                
                # Load TensorBoard data
                ea = EventAccumulator(log_path)
                ea.Reload()
                
                # Extract scalar data
                scalar_data = {}
                for tag in ea.Tags()['scalars']:
                    scalar_data[tag] = [(s.step, s.value) for s in ea.Scalars(tag)]
                
                experiment_data[exp_name] = {
                    'type': 'tensorboard',
                    'data': scalar_data,
                    'path': log_path
                }
                
                print(f"   ✅ Loaded {exp_name}: {len(scalar_data)} metrics")
                
            except Exception as e:
                print(f"   ❌ Error loading {log_path}: {e}")
    
    # Load CSV logs if available
    print("\n📊 Loading CSV logs...")
    csv_logs = glob.glob(os.path.join(CONFIG['logs_path'], '**/*.csv'), recursive=True)
    
    for csv_path in csv_logs:
        try:
            exp_name = os.path.basename(csv_path).replace('.csv', '')
            df = pd.read_csv(csv_path)
            
            experiment_data[exp_name] = {
                'type': 'csv',
                'data': df,
                'path': csv_path
            }
            
            print(f"   ✅ Loaded {exp_name}: {len(df)} rows")
            
        except Exception as e:
            print(f"   ❌ Error loading {csv_path}: {e}")
    
    # Load JSON logs if available
    print("\n📊 Loading JSON logs...")
    json_logs = glob.glob(os.path.join(CONFIG['logs_path'], '**/*.json'), recursive=True)
    
    for json_path in json_logs:
        try:
            exp_name = os.path.basename(json_path).replace('.json', '')
            
            with open(json_path, 'r') as f:
                json_data = json.load(f)
            
            experiment_data[exp_name] = {
                'type': 'json',
                'data': json_data,
                'path': json_path
            }
            
            print(f"   ✅ Loaded {exp_name}: {len(json_data)} entries")
            
        except Exception as e:
            print(f"   ❌ Error loading {json_path}: {e}")
    
    return experiment_data

# Load experiment data
print("📂 Loading experiment data...")
experiment_data = load_experiment_data()

print(f"\n✅ Loaded {len(experiment_data)} experiments")
for exp_name, exp_info in experiment_data.items():
    print(f"   {exp_name}: {exp_info['type']} ({exp_info['path']})")


In [ ]:
# Training Curves Visualization
def visualize_training_curves(experiment_data):
    """Visualize training curves for all experiments"""
    
    if not experiment_data:
        print("❌ No experiment data to visualize")
        return
    
    # Create comprehensive training curves visualization
    fig, axes = plt.subplots(2, 2, figsize=CONFIG['visualization_config']['figure_size'])
    fig.suptitle('Training Progress Comparison', fontsize=16, fontweight='bold')
    
    # Colors for different experiments
    colors = plt.cm.Set3(np.linspace(0, 1, len(experiment_data)))
    
    # 1. Training Loss
    axes[0, 0].set_title('Training Loss')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Validation Loss
    axes[0, 1].set_title('Validation Loss')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. PSNR
    axes[1, 0].set_title('PSNR')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('PSNR (dB)')
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. SSIM
    axes[1, 1].set_title('SSIM')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('SSIM')
    axes[1, 1].grid(True, alpha=0.3)
    
    # Plot data for each experiment
    for i, (exp_name, exp_info) in enumerate(experiment_data.items()):
        color = colors[i]
        
        if exp_info['type'] == 'tensorboard':
            # Handle TensorBoard data
            data = exp_info['data']
            
            # Training loss
            if 'train_loss' in data:
                steps, values = zip(*data['train_loss'])
                axes[0, 0].plot(steps, values, label=f'{exp_name} (Train)', color=color, linestyle='-')
            
            # Validation loss
            if 'val_loss' in data:
                steps, values = zip(*data['val_loss'])
                axes[0, 1].plot(steps, values, label=f'{exp_name} (Val)', color=color, linestyle='--')
            
            # PSNR
            if 'train_psnr' in data:
                steps, values = zip(*data['train_psnr'])
                axes[1, 0].plot(steps, values, label=f'{exp_name} (Train)', color=color, linestyle='-')
            
            if 'val_psnr' in data:
                steps, values = zip(*data['val_psnr'])
                axes[1, 0].plot(steps, values, label=f'{exp_name} (Val)', color=color, linestyle='--')
            
            # SSIM
            if 'train_ssim' in data:
                steps, values = zip(*data['train_ssim'])
                axes[1, 1].plot(steps, values, label=f'{exp_name} (Train)', color=color, linestyle='-')
            
            if 'val_ssim' in data:
                steps, values = zip(*data['val_ssim'])
                axes[1, 1].plot(steps, values, label=f'{exp_name} (Val)', color=color, linestyle='--')
        
        elif exp_info['type'] == 'csv':
            # Handle CSV data
            df = exp_info['data']
            
            if 'epoch' in df.columns:
                # Training loss
                if 'train_loss' in df.columns:
                    axes[0, 0].plot(df['epoch'], df['train_loss'], label=f'{exp_name} (Train)', color=color, linestyle='-')
                
                # Validation loss
                if 'val_loss' in df.columns:
                    axes[0, 1].plot(df['epoch'], df['val_loss'], label=f'{exp_name} (Val)', color=color, linestyle='--')
                
                # PSNR
                if 'train_psnr' in df.columns:
                    axes[1, 0].plot(df['epoch'], df['train_psnr'], label=f'{exp_name} (Train)', color=color, linestyle='-')
                
                if 'val_psnr' in df.columns:
                    axes[1, 0].plot(df['epoch'], df['val_psnr'], label=f'{exp_name} (Val)', color=color, linestyle='--')
                
                # SSIM
                if 'train_ssim' in df.columns:
                    axes[1, 1].plot(df['epoch'], df['train_ssim'], label=f'{exp_name} (Train)', color=color, linestyle='-')
                
                if 'val_ssim' in df.columns:
                    axes[1, 1].plot(df['epoch'], df['val_ssim'], label=f'{exp_name} (Val)', color=color, linestyle='--')
    
    # Add legends
    for ax in axes.flat:
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Training curves visualization completed!")

# Visualize training curves if data is available
if experiment_data:
    print("\n📈 Visualizing training curves...")
    visualize_training_curves(experiment_data)
else:
    print("❌ Cannot visualize training curves - no experiment data available")


In [ ]:
# Experiment Comparison and Analysis
def analyze_experiment_results(experiment_data):
    """Analyze and compare experiment results"""
    
    if not experiment_data:
        print("❌ No experiment data to analyze")
        return
    
    # Create experiment comparison dataframe
    comparison_data = []
    
    for exp_name, exp_info in experiment_data.items():
        exp_summary = {'Experiment': exp_name, 'Type': exp_info['type']}
        
        if exp_info['type'] == 'tensorboard':
            data = exp_info['data']
            
            # Extract final values
            for metric in ['train_loss', 'val_loss', 'train_psnr', 'val_psnr', 'train_ssim', 'val_ssim']:
                if metric in data:
                    final_value = data[metric][-1][1] if data[metric] else None
                    exp_summary[metric] = final_value
                else:
                    exp_summary[metric] = None
        
        elif exp_info['type'] == 'csv':
            df = exp_info['data']
            
            # Extract final values
            for metric in ['train_loss', 'val_loss', 'train_psnr', 'val_psnr', 'train_ssim', 'val_ssim']:
                if metric in df.columns:
                    final_value = df[metric].iloc[-1] if not df[metric].empty else None
                    exp_summary[metric] = final_value
                else:
                    exp_summary[metric] = None
        
        comparison_data.append(exp_summary)
    
    # Create comparison dataframe
    df_comparison = pd.DataFrame(comparison_data)
    
    # Display comparison table
    print("\n📊 Experiment Results Comparison:")
    print(df_comparison.to_string(index=False, float_format='%.6f'))
    
    # Create visualization
    if len(comparison_data) > 1:
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        fig.suptitle('Experiment Results Comparison', fontsize=16, fontweight='bold')
        
        # Metrics to visualize
        metrics = [
            ('val_loss', 'Validation Loss (Lower is Better)', axes[0, 0]),
            ('val_psnr', 'Validation PSNR (Higher is Better)', axes[0, 1]),
            ('val_ssim', 'Validation SSIM (Higher is Better)', axes[0, 2]),
            ('train_loss', 'Training Loss (Lower is Better)', axes[1, 0]),
            ('train_psnr', 'Training PSNR (Higher is Better)', axes[1, 1]),
            ('train_ssim', 'Training SSIM (Higher is Better)', axes[1, 2])
        ]
        
        for metric, title, ax in metrics:
            if metric in df_comparison.columns:
                # Filter out None values
                valid_data = df_comparison[df_comparison[metric].notna()]
                
                if not valid_data.empty:
                    bars = ax.bar(valid_data['Experiment'], valid_data[metric], 
                                color=plt.cm.Set3(np.linspace(0, 1, len(valid_data))))
                    ax.set_title(title)
                    ax.set_ylabel(metric.replace('_', ' ').title())
                    ax.grid(True, alpha=0.3)
                    
                    # Add value labels on bars
                    for bar, value in zip(bars, valid_data[metric]):
                        height = bar.get_height()
                        ax.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                               f'{value:.3f}', ha='center', va='bottom')
                else:
                    ax.text(0.5, 0.5, f'No data for {metric}', ha='center', va='center', transform=ax.transAxes)
                    ax.set_title(title)
            else:
                ax.text(0.5, 0.5, f'No data for {metric}', ha='center', va='center', transform=ax.transAxes)
                ax.set_title(title)
        
        plt.tight_layout()
        plt.show()
    
    # Find best experiments
    print("\n🏆 Best Experiments by Metric:")
    
    for metric in ['val_psnr', 'val_ssim', 'val_loss']:
        if metric in df_comparison.columns:
            valid_data = df_comparison[df_comparison[metric].notna()]
            
            if not valid_data.empty:
                if metric == 'val_loss':
                    # Lower is better
                    best_exp = valid_data.loc[valid_data[metric].idxmin(), 'Experiment']
                    best_value = valid_data[metric].min()
                else:
                    # Higher is better
                    best_exp = valid_data.loc[valid_data[metric].idxmax(), 'Experiment']
                    best_value = valid_data[metric].max()
                
                print(f"   {metric}: {best_exp} ({best_value:.6f})")
    
    return df_comparison

# Analyze experiment results if data is available
if experiment_data:
    print("\n📊 Analyzing experiment results...")
    comparison_df = analyze_experiment_results(experiment_data)
else:
    print("❌ Cannot analyze results - no experiment data available")


In [ ]:
# Export Experiment Results
def export_experiment_analysis(experiment_data, comparison_df, output_path):
    """Export experiment analysis results"""
    
    if not experiment_data:
        print("❌ No experiment data to export")
        return
    
    print(f"💾 Exporting experiment analysis to {output_path}...")
    
    # Create output directories
    os.makedirs(output_path, exist_ok=True)
    os.makedirs(os.path.join(output_path, 'plots'), exist_ok=True)
    os.makedirs(os.path.join(output_path, 'data'), exist_ok=True)
    
    # Export comparison dataframe
    if comparison_df is not None:
        comparison_path = os.path.join(output_path, 'data', 'experiment_comparison.csv')
        comparison_df.to_csv(comparison_path, index=False)
        print(f"   ✅ Exported comparison data: {comparison_path}")
    
    # Export individual experiment data
    for exp_name, exp_info in experiment_data.items():
        exp_output_path = os.path.join(output_path, 'data', f'{exp_name}_data')
        os.makedirs(exp_output_path, exist_ok=True)
        
        if exp_info['type'] == 'tensorboard':
            # Export TensorBoard data as CSV
            data = exp_info['data']
            for metric, values in data.items():
                if values:
                    df_metric = pd.DataFrame(values, columns=['step', 'value'])
                    metric_path = os.path.join(exp_output_path, f'{metric}.csv')
                    df_metric.to_csv(metric_path, index=False)
            
            print(f"   ✅ Exported {exp_name} TensorBoard data")
        
        elif exp_info['type'] == 'csv':
            # Export CSV data
            df = exp_info['data']
            csv_path = os.path.join(exp_output_path, f'{exp_name}_data.csv')
            df.to_csv(csv_path, index=False)
            print(f"   ✅ Exported {exp_name} CSV data")
        
        elif exp_info['type'] == 'json':
            # Export JSON data
            json_data = exp_info['data']
            json_path = os.path.join(exp_output_path, f'{exp_name}_data.json')
            with open(json_path, 'w') as f:
                json.dump(json_data, f, indent=2)
            print(f"   ✅ Exported {exp_name} JSON data")
    
    # Create summary report
    summary_report = {
        'export_timestamp': datetime.now().isoformat(),
        'total_experiments': len(experiment_data),
        'experiment_types': list(set(exp_info['type'] for exp_info in experiment_data.values())),
        'available_metrics': CONFIG['metrics_to_track'],
        'export_path': output_path
    }
    
    summary_path = os.path.join(output_path, 'experiment_summary.json')
    with open(summary_path, 'w') as f:
        json.dump(summary_report, f, indent=2)
    
    print(f"   ✅ Exported summary report: {summary_path}")
    print(f"\n✅ Export completed!")
    print(f"   Output directory: {output_path}")
    print(f"   Data: {os.path.join(output_path, 'data')}")
    print(f"   Plots: {os.path.join(output_path, 'plots')}")

# Export results if available
if experiment_data:
    print("\n💾 Exporting experiment analysis...")
    export_experiment_analysis(experiment_data, 
                              comparison_df if 'comparison_df' in locals() else None, 
                              CONFIG['results_path'])
else:
    print("❌ Cannot export results - no experiment data available")


## Summary and Key Insights

### Experiment Tracking & Comparison Results:

1. **Data Loading**: ✅ Successfully loaded experiment data from multiple sources
2. **Training Curves**: ✅ Comprehensive visualization of training progress
3. **Model Comparison**: ✅ Detailed comparison of different model architectures
4. **Export Capabilities**: ✅ Multiple output formats for further analysis

### Key Findings:

1. **Experiment Management**:
   - Multiple data sources (TensorBoard, CSV, JSON) supported
   - Comprehensive experiment tracking and organization
   - Automated data loading and processing

2. **Training Analysis**:
   - Training curves show convergence patterns
   - Validation metrics indicate model performance
   - Loss curves reveal training stability

3. **Model Comparison**:
   - Side-by-side comparison of different architectures
   - Performance metrics guide model selection
   - Statistical analysis reveals model strengths

### Recommendations:

1. **Experiment Organization**: Use consistent naming and logging conventions
2. **Data Management**: Store experiment data in structured formats
3. **Visualization**: Use comprehensive visualization for result analysis
4. **Model Selection**: Consider multiple metrics for model selection

### Next Steps:
- Use the metrics analysis notebook for comprehensive evaluation
- Run the inference notebook for production deployment
- Use the preprocessing notebook for data quality assessment

---
*This notebook provides a comprehensive experiment tracking and comparison framework for SAR image colorization. The insights gained will guide model selection and improvement strategies.*
